In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Check GPU name
!nvidia-smi


TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Wed May 28 19:31:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                              

In [ ]:
!pip install h5py imageio matplotlib scikit-image tqdm


In [ ]:
!apt-get install libhdf4-0 libhdf4-dev -y
!pip install pyhdf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  default-libmysqlclient-dev libaom-dev libarmadillo-dev libarpack2-dev
  libblosc-dev libcfitsio-dev libdav1d-dev libde265-dev libfreexl-dev
  libfyba-dev libgeos-dev libgeotiff-dev libgif-dev libheif-dev libjson-c-dev
  libkml-dev libkmlconvenience1 libkmlregionator1 libkmlxsd1 liblz4-dev
  libminizip-dev libmysqlclient-dev libnetcdf-dev libodbccr2 libogdi-dev
  libopenjp2-7-dev libpoppler-dev libpoppler-private-dev libpq-dev libproj-dev
  libqhull-dev libqhull8.0 libqhullcpp8.0 librttopo-dev libspatialite-dev
  libsqlite3-dev libsuperlu-dev liburiparser-dev libwebp-dev libx265-dev
  libxerces-c-dev unixodbc-dev
Use 'apt autoremove' to remove them.
Suggested packages:
  libhdf4-doc hdf4-tools
The following packages will be REMOVED:
  libgdal-dev libhdf4-alt-dev
The following NEW packages will be installed:
  l

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define paths
input_dir = "/content/drive/MyDrive/MOD09GQ_061-20250521_065631" # change if your folder name is different
output_hr_dir = "/content/drive/MyDrive/SRGAN_data/HR"
output_lr_dir = "/content/drive/MyDrive/SRGAN_data/LR"

import os
os.makedirs(output_hr_dir, exist_ok=True)
os.makedirs(output_lr_dir, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from pyhdf.SD import SD, SDC

input_dir = "/content/drive/MyDrive/MOD09GA_061-20250524_135742"
output_hr_dir = "/content/drive/MyDrive/SRGANN_data/HR"
output_lr_dir = "/content/drive/MyDrive/SRGANN_data/LR"

os.makedirs(output_hr_dir, exist_ok=True)
os.makedirs(output_lr_dir, exist_ok=True)

def process_modis_file(hdf_path, prefix):
    try:
        hdf = SD(hdf_path, SDC.READ)
        raw_red = hdf.select('sur_refl_b01_1')[:]
        red = np.where(raw_red == -28672, np.nan, raw_red).astype(np.float32)
        red = red * 0.0001

        # ✅ Apply percentile stretching (2nd to 98th)
        p2 = np.nanpercentile(red, 2)
        p98 = np.nanpercentile(red, 98)
        red = np.clip((red - p2) / (p98 - p2), 0, 1)
        red = np.nan_to_num(red, nan=0.0)

        red_img = (red * 255).astype(np.uint8)
        h, w = red_img.shape
        crop = red_img[h//2 - 512:h//2 + 512, w//2 - 512:w//2 + 512]

        pid = 0
        for i in range(0, 1024, 256):
            for j in range(0, 1024, 256):
                patch = crop[i:i+256, j:j+256]
                if patch.shape == (256, 256) and patch.mean() > 10:
                    hr_img = Image.fromarray(patch)
                    lr_img = hr_img.resize((64, 64), Image.BICUBIC)

                    hr_path = os.path.join(output_hr_dir, f"{prefix}_{pid}_HR.png")
                    lr_path = os.path.join(output_lr_dir, f"{prefix}_{pid}_LR.png")

                    hr_img.save(hr_path)
                    lr_img.save(lr_path)
                    pid += 1
    except Exception as e:
        print(f"❌ Error processing {hdf_path}: {e}")

file_list = [f for f in os.listdir(input_dir) if f.endswith('.hdf')]
for idx, file in enumerate(tqdm(file_list, desc="🔄 Processing HDF files")):
    process_modis_file(os.path.join(input_dir, file), f"img{idx}")


🔄 Processing HDF files: 100%|██████████| 100/100 [04:42<00:00,  2.82s/it]


In [ ]:
# ✅ STEP 1: Import Libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
# ✅ STEP 2: Load Dataset (LR and HR images)
def load_image_pair(lr_path, hr_path):
    lr = load_img(lr_path, color_mode='grayscale', target_size=(64, 64))
    hr = load_img(hr_path, color_mode='grayscale', target_size=(256, 256))
    lr = img_to_array(lr) / 255.0
    hr = img_to_array(hr) / 255.0
    return lr, hr

In [ ]:
def load_dataset(lr_dir, hr_dir):
    lr_images = sorted(glob(os.path.join(lr_dir, '*.png')))
    hr_images = sorted(glob(os.path.join(hr_dir, '*.png')))
    dataset = [load_image_pair(lr, hr) for lr, hr in zip(lr_images, hr_images)]
    return zip(*dataset)  # returns (LRs, HRs)

In [ ]:
# ✅ STEP 3: Define the Generator

def residual_block(x):
    res = layers.Conv2D(64, 3, padding='same')(x)
    res = layers.BatchNormalization()(res)
    res = layers.PReLU(shared_axes=[1, 2])(res)
    res = layers.Conv2D(64, 3, padding='same')(res)
    res = layers.BatchNormalization()(res)
    return layers.add([x, res])

def build_generator():
    inputs = layers.Input(shape=(64, 64, 1))
    x = layers.Conv2D(64, 9, padding='same')(inputs)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    res = x
    for _ in range(16):
        res = residual_block(res)
    x = layers.Conv2D(64, 3, padding='same')(res)
    x = layers.BatchNormalization()(x)
    x = layers.add([x, res])
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    output = layers.Conv2D(1, 9, activation='tanh', padding='same')(x)
    return models.Model(inputs, output, name="generator")

In [ ]:
# ✅ STEP 4: Define the Discriminator

def build_discriminator():
    def conv_block(x, filters, strides):
        x = layers.Conv2D(filters, 3, strides=strides, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(0.2)(x)
        return x

    inputs = layers.Input(shape=(256, 256, 1))
    x = layers.Conv2D(64, 3, strides=1, padding='same')(inputs)
    x = layers.LeakyReLU(0.2)(x)
    for filters, strides in zip([64, 128, 128, 256, 256, 512, 512], [2, 1, 2, 1, 2, 1, 2]):
        x = conv_block(x, filters, strides)
    x = layers.Flatten()(x)
    x = layers.Dense(1024)(x)
    x = layers.LeakyReLU(0.2)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, output, name="discriminator")

In [ ]:
# ✅ STEP 5: VGG Feature Extractor (for perceptual loss)
def build_vgg():
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    vgg.trainable = False
    model = models.Model(inputs=vgg.input, outputs=vgg.get_layer('block5_conv4').output)
    return model

In [ ]:
# ✅ STEP 6: Define Loss Functions
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()
mse = tf.keras.losses.MeanSquaredError()

In [ ]:
# ✅ STEP 7: PSNR Metric
def psnr(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

In [ ]:
# ✅ STEP 8: Training Setup
lr_images, hr_images = load_dataset('/content/drive/MyDrive/SRGANN_data/LR','/content/drive/MyDrive/SRGANN_data/HR')

lr_images = np.array(lr_images).astype('float32')
hr_images = np.array(hr_images).astype('float32')

generator = build_generator()
discriminator = build_discriminator()
vgg = build_vgg()

vgg.trainable = False

# Optimizers
g_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# ✅ STEP 9: Training Loop
@tf.function
def train_step(lr, hr):
    valid = tf.ones((lr.shape[0], 1))
    fake = tf.zeros((lr.shape[0], 1))

    with tf.GradientTape() as tape:
        fake_hr = generator(lr, training=True)
        real_output = discriminator(hr, training=True)
        fake_output = discriminator(fake_hr, training=True)

        perceptual_loss = mse(vgg(tf.image.grayscale_to_rgb(hr)), vgg(tf.image.grayscale_to_rgb(fake_hr)))
        content_loss = mse(hr, fake_hr)
        adv_loss = binary_cross_entropy(valid, fake_output)
        g_loss = 1e-3 * adv_loss + 0.006 * perceptual_loss + content_loss

    grads = tape.gradient(g_loss, generator.trainable_variables)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_variables))

    with tf.GradientTape() as tape:
        real_output = discriminator(hr, training=True)
        fake_output = discriminator(generator(lr, training=False), training=True)
        d_loss = binary_cross_entropy(valid, real_output) + binary_cross_entropy(fake, fake_output)

    grads = tape.gradient(d_loss, discriminator.trainable_variables)
    d_optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))
    return g_loss, d_loss


In [ ]:
EPOCHS = 10
BATCH_SIZE = 8
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    g_loss_total, d_loss_total = 0, 0
    for i in range(0, len(lr_images), BATCH_SIZE):
        lr_batch = lr_images[i:i+BATCH_SIZE]
        hr_batch = hr_images[i:i+BATCH_SIZE]
        g_loss, d_loss = train_step(lr_batch, hr_batch)

        g_loss_total += g_loss
        d_loss_total += d_loss

        # ✅ Add a batch log every 10 batches
        if i // BATCH_SIZE % 10 == 0:
            print(f"Batch {i//BATCH_SIZE + 1}/{len(lr_images) // BATCH_SIZE} | "
                  f"G Loss: {g_loss:.4f}, D Loss: {d_loss:.4f}")

    print(f"Epoch {epoch+1} complete → Generator loss: {g_loss_total:.4f}, Discriminator loss: {d_loss_total:.4f}")



Epoch 1/10
Batch 1/160 | G Loss: 0.1624, D Loss: 1.5094
Batch 11/160 | G Loss: 0.0894, D Loss: 0.0007
Batch 21/160 | G Loss: 0.0490, D Loss: 0.0001
Batch 31/160 | G Loss: 0.0345, D Loss: 0.0000
Batch 41/160 | G Loss: 0.0291, D Loss: 0.0000
Batch 51/160 | G Loss: 0.0465, D Loss: 0.0011
Batch 61/160 | G Loss: 0.0481, D Loss: 0.0000
Batch 71/160 | G Loss: 0.0326, D Loss: 0.0000
Batch 81/160 | G Loss: 0.0330, D Loss: 0.0000
Batch 91/160 | G Loss: 0.0193, D Loss: 0.0000
Batch 101/160 | G Loss: 0.0157, D Loss: 0.0000
Batch 111/160 | G Loss: 0.0312, D Loss: 0.0000
Batch 121/160 | G Loss: 0.0107, D Loss: 0.0000
Batch 131/160 | G Loss: 0.0230, D Loss: 0.0000
Batch 141/160 | G Loss: 0.0149, D Loss: 0.0000
Batch 151/160 | G Loss: 0.0251, D Loss: 0.0000
Batch 161/160 | G Loss: 0.0192, D Loss: 0.0000
Epoch 1 complete → Generator loss: 6.8364, Discriminator loss: 107.7633

Epoch 2/10
Batch 1/160 | G Loss: 0.0545, D Loss: 0.0000
Batch 11/160 | G Loss: 0.0104, D Loss: 0.0000
Batch 21/160 | G Loss: 0.

from google.colab import drive
drive.mount('/content/drive')
